In [ ]:
# 🛠️ Step 1: Authenticate
from google.colab import auth
auth.authenticate_user()

# 📦 Imports
from google.cloud import bigquery
from google.auth import default
import pandas as pd
import json
import os

# 🧠 Step 2: Set project ID
PROJECT_ID = 'INSERT_BIGQUERY_PROJECT_ID_HERE'  # Replace with your actual project

# 💬 Step 3: Optional dataset selector
dataset_input = input("Enter a specific Dataset ID to scan (or leave blank to scan ALL datasets): ")
target_dataset_id = dataset_input.strip()

# 💬 Step 4: Choose export format
export_format_input = input("Choose export format ('csv' or 'json'): ")
export_format = export_format_input.strip().lower()
assert export_format in ['csv', 'json'], "Please enter 'csv' or 'json'"

# 🔌 Step 5: Initialize client
creds, _ = default()
client = bigquery.Client(project=PROJECT_ID, credentials=creds)

# 📡 Step 6: Extract schema
print(f"\n🔍 Scanning BigQuery project: {PROJECT_ID}")
schema_data = []

datasets = client.list_datasets()

for dataset in datasets:
    dataset_id = dataset.dataset_id
    full_dataset = f"{PROJECT_ID}.{dataset_id}"

    if target_dataset_id and dataset_id != target_dataset_id:
        continue  # Skip unrelated datasets if one is specified

    print(f"\n📁 Dataset: {full_dataset}")
    tables = client.list_tables(full_dataset)

    if not tables:
        print("  (No tables found)")
        continue

    for table in tables:
        full_table_id = f"{full_dataset}.{table.table_id}"
        print(f"  📄 Table: {table.table_id}")
        table_ref = client.get_table(full_table_id)

        for field in table_ref.schema:
            schema_entry = {
                "project_id": PROJECT_ID,
                "dataset_id": dataset_id,
                "table_id": table.table_id,
                "field_name": field.name,
                "field_type": field.field_type,
                "field_mode": field.mode
            }
            schema_data.append(schema_entry)
            print(f"    - {field.name} ({field.field_type}, {field.mode})")

# 💾 Step 7: Export schema
df = pd.DataFrame(schema_data)
filename = f"bigquery_schema.{export_format}"
filepath = f"/content/{filename}"

if export_format == "csv":
    df.to_csv(filepath, index=False)
elif export_format == "json":
    with open(filepath, "w") as f:
        json.dump(schema_data, f, indent=2)

print(f"\n✅ Schema exported to '{filename}'")

# 📤 Step 8: Choose destination
destination_input = input("Do you want to save to Google Drive? (yes/no): ").strip().lower()

if destination_input == "yes":
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        drive_path = f"/content/drive/My Drive/{filename}"
        os.rename(filepath, drive_path)
        print(f"✅ File saved to Google Drive at: {drive_path}")
    except Exception as e:
        print(f"⚠️ Failed to save to Drive: {e}")
else:
    from google.colab import files
    files.download(filepath)
    print("📎 File download initiated.")
